In [1]:
n0 = 6000
n1 = n0 + 949

In [2]:
import pandas as pd
import random
import numpy as np

In [3]:
authorship = pd.read_csv('G1_mentee_authorship.csv')

In [4]:
target_author = pd.read_csv('target_author.csv')['author_id'].tolist()

In [5]:
len(target_author)

6949

In [6]:
authorship.head(5)

,work_id,author_position,author_id,institution_id,first_pub_year,last_pub_year,career_len,publication_year,scientific_age
0,W1033085884,first,A3022368640,I5388228,2006,2006,0,2006,0
1,W1033085884,middle,A2613136029,I5388228,2003,2010,7,2006,3
2,W1033085884,middle,A2044805072,I5388228,2006,2006,0,2006,0
3,W1033085884,middle,A2886369024,I5388228,2005,2006,1,2006,1
4,W1033085884,middle,A1016359704,I5388228,2006,2006,0,2006,0


In [7]:
len(set(authorship.work_id))

903183

In [8]:
inst_completeness = pd.read_csv('G1_mentee_inst_completeness.csv')

In [9]:
target_works1 = set(inst_completeness['work_id'][inst_completeness['ratio_inst.']>0])
len(target_works1)

751272

In [10]:
authorship = authorship[authorship.work_id.isin(target_works1)]

In [11]:
def group_info(author_i,df,backward_window,scientific_age_window):
    auship = df
    
    # Method2
    #mentor's all last-author publications
    works_i = auship['work_id'][(auship.author_id==author_i)&(auship.author_position=='last')]
    

    auship_i = auship[auship.work_id.isin(works_i)].sort_values(by='publication_year')
    # print('len(auship_i) : ',len(auship_i))

    auship_null = auship_i[~auship_i.institution_id.isnull()]

    # print('the ratio of with institution : ','{:.0%}'.format(len(auship_null)/len(auship_i)))
    
    years_i = auship_i.publication_year
    try:
        start_year = min(years_i)
        end_year = max(years_i)
    except:
        start_year = 2021
        end_year = 2022
    # print('start_year : ',start_year)
    # print('end_year : ',end_year)
    
    Group = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
   
    for current_year in range(start_year,end_year+1):
        # print('current_year : ',current_year)
        auship_i_back = auship_i[(auship_i.publication_year>=current_year-backward_window)&
                                (auship_i.publication_year<=current_year)]
        
        auship_i_back =  auship_i_back.reset_index().drop(columns=['index'])
        #new consideration : current_scientific_age
        csa = np.array(auship_i_back['first_pub_year'] - current_year)*-1
        auship_i_back['current_scientific_age']  = csa

        auship_i_back_notnull = auship_i_back[~auship_i_back.institution_id.isnull()]
        
        
        if len(auship_i_back)==0:
            ratio_institution = 0
        else:
            ratio_institution = len(auship_i_back_notnull)/len(auship_i_back)
            
            # print('the ratio of with institution : ','{:.0%}'.format(ratio_institution))
        
        
        # ratio_institution = len(auship_i_back_notnull)/len(auship_i_back)
        # print('the ratio of with institution : ','{:.0%}'.format(ratio_institution))

        if not ratio_institution==0:
            try:
                institution_i = auship_i_back['institution_id'][auship_i_back.author_id==author_i].mode().iloc[0]
                # print('institution_i : ',institution_i)

                group_i = auship_i_back[(auship_i_back.institution_id==institution_i)
                                       &(auship_i_back.current_scientific_age<=scientific_age_window) # see above: current_scientific_age
                                       &(auship_i_back.author_id!=author_i)]

                group_size = len(set(group_i.author_id))
            # print('group_size : ',group_size)
            except:
                group_size = 0
            if group_size>0:
                group_i = group_i.assign(group_size = group_size)
                group_i = group_i.assign(mentor_id = author_i)
                group_i = group_i.assign(current_year = current_year)
                group_i = group_i.assign(ratio_institution = ratio_institution)
            else:
                group_i = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
        else:
            group_i = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
        Group = pd.concat((Group,group_i))
    return Group

In [12]:
backward_window = 5
scientific_age_window = 10
Data = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])

for i in range(n0,n1):
    author_i = target_author[i]
    data = group_info(author_i,authorship,backward_window,scientific_age_window)
    Data = pd.concat((Data,data))
    if i%10==0:
        print(i)

Data['current_scientific_age'] = Data['current_year'] - Data['first_pub_year']
Data.to_csv(f'groupsize/author_{n0}_{n1}.csv',index=0)

6000
6010
6020
6030
6040
6050
6060
6070
6080
6090
6100
6110
6120
6130
6140
6150
6160
6170
6180
6190
6200
6210
6220
6230
6240
6250
6260
6270
6280
6290
6300
6310
6320
6330
6340
6350
6360
6370
6380
6390
6400
6410
6420
6430
6440
6450
6460
6470
6480
6490
6500
6510
6520
6530
6540
6550
6560
6570
6580
6590
6600
6610
6620
6630
6640
6650
6660
6670
6680
6690
6700
6710
6720
6730
6740
6750
6760
6770
6780
6790
6800
6810
6820
6830
6840
6850
6860
6870
6880
6890
6900
6910
6920
6930
6940
